In [32]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# 경로 설정
base_model_path = "Qwen/Qwen3-4B-Instruct-2507"
adapter_path = "./qwen3-4b-text-to-sql-ko/checkpoint-750"
merged_model_path = "./output_dir_checkpoint-750"

# 디바이스 설정
device_arg = {"device_map": "auto"}

# 베이스 모델 로드
print(f"Loading base model from: {base_model_path}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    return_dict=True,
    torch_dtype=torch.float16,
    **device_arg
)

# LoRA 어댑터 로드 및 병합
print(f"Loading and merging PEFT from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path, **device_arg)
model = model.merge_and_unload()

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# 저장
print(f"Saving merged model to: {merged_model_path}")
model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)
print("✅ 모델과 토크나이저 저장 완료")

Loading base model from: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading and merging PEFT from: ./qwen3-4b-text-to-sql-ko/checkpoint-750
Saving merged model to: ./output_dir_checkpoint-750
✅ 모델과 토크나이저 저장 완료


In [33]:
from huggingface_hub import HfApi
api = HfApi()

username = "iamjoon"

In [34]:
MODEL_NAME = 'qwen3-4b-text-to-sql-ko-checkpoint-750'

In [35]:
api.create_repo(
    token="hf_여러분의 키 값",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

RepoUrl('https://huggingface.co/iamjoon/qwen3-4b-text-to-sql-ko-checkpoint-750', endpoint='https://huggingface.co', repo_type='model', repo_id='iamjoon/qwen3-4b-text-to-sql-ko-checkpoint-750')

In [36]:
api.upload_folder(
    token="hf_여러분의 키 값",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir_checkpoint-750",
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t_dir_checkpoint-750/tokenizer.json: 100%|##########| 11.4MB / 11.4MB            

  ...50/model-00001-of-00002.safetensors:   1%|1         | 58.6MB / 4.97GB            

  ...50/model-00002-of-00002.safetensors:   2%|1         | 50.3MB / 3.08GB            

CommitInfo(commit_url='https://huggingface.co/iamjoon/qwen3-4b-text-to-sql-ko-checkpoint-750/commit/f18277ea793e41737b1a75bd4e29167f83d1654b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f18277ea793e41737b1a75bd4e29167f83d1654b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iamjoon/qwen3-4b-text-to-sql-ko-checkpoint-750', endpoint='https://huggingface.co', repo_type='model', repo_id='iamjoon/qwen3-4b-text-to-sql-ko-checkpoint-750'), pr_revision=None, pr_num=None)

In [37]:
%rm -r output_dir_checkpoint-750